#### Monday, April 29, 2024

mamba activate llama3

This notebook is from the video [Fine Tune Llama 3 using ORPO](https://www.youtube.com/watch?v=nPIGVaYPQAg).

My expectation is this notebook will run because 'Fine_tune_Llama_3_with_ORPO.ipynb' also ran ... let's find out, shall we ... 

It really looks like this guy stole the code from 'Fine_tune_Llama_3_with_ORPO.ipynb' and just used a different dataset ... what a douche.

So it looks like the training DOES work, but it was going to take a real long time (5+ hours ?) to run, so I aborted.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# This is the code needed to get the other notebook to work, so gonna try it here ... 
#   Using RTX 4000 series doesn't support faster communication broadband via P2P or IB. 
#   Please set `NCCL_P2P_DISABLE="1"` and `NCCL_IB_DISABLE="1" or use `accelerate launch` which will do this automatically.

import os

os.environ["NCCL_P2P_DISABLE"]="1"
os.environ["NCCL_IB_DISABLE"]="1"

In [3]:
# !pip install -U transformers datasets accelerate peft trl bitsandbytes wandb

In [4]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
# from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

In [5]:
# wb_token = userdata.get('wandb')
wb_token = os.environ["WANDB_API_KEY"] 
wandb.login(key=wb_token)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robkayinto. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/rob/.netrc


True

In [6]:
torch.cuda.get_device_capability()[0]

8

In [7]:
# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     attn_implementation = "flash_attention_2"
#     torch_dtype = torch.bfloat16
# else:
#     attn_implementation = "eager"
#     torch_dtype = torch.float16

attn_implementation = "flash_attention_2"
torch_dtype = torch.bfloat16

In [6]:
#from huggingface_hub import notebook_login
# notebook_login()

In [8]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "OrpoLlama3-8B-FT"

In [9]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

In [10]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)


In [11]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

In [14]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 9m 18.8s

Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
dataset

[Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 61135
 }),
 Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 2000
 })]

In [16]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

In [17]:
# Calculate the number of test samples based on the new training sample size
train_samples = 5000
original_train_samples = 61135
test_samples = int((2000 / original_train_samples) * train_samples)

In [18]:
# Shuffle and select a subset from both train and test sets
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))

In [19]:
print(train_subset)
print(test_subset)


Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
})
Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})


In [21]:
import multiprocessing

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

In [22]:
dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

Map (num_proc=8):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [23]:
dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

Map (num_proc=8):   0%|          | 0/163 [00:00<?, ? examples/s]

In [24]:
print(dataset)

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


In [27]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=1000,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./resultsFT/",
)

In [28]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    tokenizer=tokenizer,
)

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [29]:
# This grinds the 4090 at around 99% usage, 420w, 22gb ... it really puts a load on this thing! But it works! Nice!

# I aborted this training cuz it was gonna take too long to run ... 52m, 14%, 139/1000 .... it seemed to stay stuck in this stage so I aborted ...

trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 5.4286, 'grad_norm': 74.62857055664062, 'learning_rate': 8e-07, 'rewards/chosen': -0.1339951753616333, 'rewards/rejected': -0.12913674116134644, 'rewards/accuracies': 0.25, 'rewards/margins': -0.004858411382883787, 'logps/rejected': -1.291367530822754, 'logps/chosen': -1.339951515197754, 'logits/rejected': -1.787912130355835, 'logits/chosen': -1.8489067554473877, 'nll_loss': 5.356834411621094, 'log_odds_ratio': -0.7173362970352173, 'log_odds_chosen': -0.02114342898130417, 'epoch': 0.0}
{'loss': 6.0944, 'grad_norm': 85.59339141845703, 'learning_rate': 1.6e-06, 'rewards/chosen': -0.1236344575881958, 'rewards/rejected': -0.12871555984020233, 'rewards/accuracies': 0.5, 'rewards/margins': 0.0050811017863452435, 'logps/rejected': -1.2871556282043457, 'logps/chosen': -1.236344575881958, 'logits/rejected': -1.6551487445831299, 'logits/chosen': -1.978860855102539, 'nll_loss': 6.02786922454834, 'log_odds_ratio': -0.6648973822593689, 'log_odds_chosen': 0.11178050935268402, 'epoch': 0.0}


  0%|          | 0/82 [00:00<?, ?it/s]

{'eval_loss': 0.9303590655326843, 'eval_runtime': 101.8635, 'eval_samples_per_second': 1.6, 'eval_steps_per_second': 0.805, 'eval_rewards/chosen': -0.07260670512914658, 'eval_rewards/rejected': -0.07715044915676117, 'eval_rewards/accuracies': 0.47560974955558777, 'eval_rewards/margins': 0.00454374123364687, 'eval_logps/rejected': -0.7715045213699341, 'eval_logps/chosen': -0.7260671257972717, 'eval_logits/rejected': -2.206099033355713, 'eval_logits/chosen': -1.9944219589233398, 'eval_nll_loss': 0.8562071323394775, 'eval_log_odds_ratio': -0.7192689180374146, 'eval_log_odds_chosen': 0.05539192631840706, 'epoch': 0.32}
{'loss': 0.832, 'grad_norm': 0.3767867684364319, 'learning_rate': 6.456565656565656e-06, 'rewards/chosen': -0.08055874705314636, 'rewards/rejected': -0.08539722114801407, 'rewards/accuracies': 0.375, 'rewards/margins': 0.004838474560528994, 'logps/rejected': -0.8539721965789795, 'logps/chosen': -0.8055874705314636, 'logits/rejected': -2.156276226043701, 'logits/chosen': -2.5

KeyboardInterrupt: 

In [ ]:
trainer.save_model(new_model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


## How to Merge LoRA Adapater

In [ ]:
del trainer, model
gc.collect()
torch.cuda.empty_cache()

#reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict = True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model, tokenizer = setup_chat_format(model, tokenizer)

#Merge
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)